In [1]:
import sys
import secretflow as sf
sys.path.append("/home/yejj/GradExperiments/chapter3")
from utils.model import create_model

from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import PlainAggregator

sf.shutdown()
# init nodes
# sf.init(nodes, address="local")
sf.init(['alice', 'bob','server'], address="local")
# init PYU device
alice, bob, agg_server = sf.PYU("alice"), sf.PYU("bob"), sf.PYU("server")


from secretflow.utils.simulation.datasets import load_mnist

(x_train, y_train), (x_test, y_test) = load_mnist(parts=[alice, bob], normalized_x=True, categorical_y=True)
# print(y_test.partition_shape())

2024-02-24 21:22:04.014492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-24 21:22:05.053557: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-24 21:22:05.053789: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-24 21:22:05.053803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
model = create_model(input_shape, num_classes, "lenet")

plain_aggregator = PlainAggregator(agg_server)

fed_plain_model = FLModel(server=agg_server,
                    device_list=[alice, bob],
                    model=model,
                    aggregator=plain_aggregator,
                    strategy="fed_avg_w",
                    backend = "tensorflow")

from secretflow.ml.nn.callbacks.early_stopping import EarlyStoppingEpoch

earlystop_callback = EarlyStoppingEpoch(
		monitor='val_loss', min_delta=0.01, mode='min',
        patience=1)

history = fed_plain_model.fit(x_train,
                    y_train,
                    validation_data=(x_test, y_test),
                    epochs=1,
                    sampler_method="batch",
                    batch_size=128,
                    aggregate_freq=1,
                    callbacks=earlystop_callback)
# plain_time = time.time() - start_time
# print("cost time : ", time.time()-start_time)
print(history)

2024-02-24 21:22:13.725382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-24 21:22:13.725443: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.fl.backend.tensorflow.strategy.fed_avg_w.PYUFedAvgW'> with party bob.
INFO:root:FL Train Params: {'x': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7f6ec073f250>, PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7f6ec0707880>}, partition_way=<PartitionWay.HORIZONTAL: 'horizontal'>), 'y': FedNdarray(partitions={PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at

(pid=50844) 2024-02-24 21:22:16.020673: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=50885) 2024-02-24 21:22:16.216807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=50844) 2024-02-24 21:22:17.142925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
(pid=50844) 2024-02-24 21:22:17.143092: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: 

{'global_history': {'loss': [0.483213], 'accuracy': [0.87053335], 'val_loss': [0.1297757], 'val_accuracy': [0.962]}, 'local_history': {'alice_loss': [0.48317063], 'alice_accuracy': [0.87023336], 'alice_val_loss': [0.17485225], 'alice_val_accuracy': [0.9468], 'bob_loss': [0.48325545], 'bob_accuracy': [0.87083334], 'bob_val_loss': [0.08469918], 'bob_val_accuracy': [0.9772]}}


In [3]:
# 统计收敛轮次 and 绘制loss下降图
from secretflow.device import reveal
import numpy as np

pred_fed_y = fed_plain_model.predict(x_test, batch_size=128)

In [6]:
# 拼接y_test
original_y = np.array([])
pred_y = np.array([])

for device in [alice, bob]:
    temp = y_test.partitions[device]
    original_y = np.concatenate((original_y, np.argmax(np.array(reveal(temp)), axis=1)))

    temp1 = pred_fed_y[device]
    pred_y = np.concatenate((pred_y, np.argmax(np.array(reveal(temp1)), axis=1)))

# 拼接y_pred
# for device, y in pred_fed_y.items():
#     pred_fed_y[device] = np.argmax(np.array(reveal(y)), axis=1)

from sklearn.metrics import f1_score, accuracy_score
f1_micro = f1_score(original_y, pred_y, average='micro')
print(f1_micro)

(10000,)
(10000,)
0.962


In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(original_y, pred_y)
print(accuracy)

0.962
